# <center> Learning the topology of a Bayesian Network from a database of cases using the K2 algorithm 

## Imports


In [16]:
library(dplyr)
library(Rgraphviz)

ERROR: Error in library(Rgraphviz): there is no package called ‘Rgraphviz’


The aim of this project is to build a probabilistic network from the following database called D:

In [2]:
# input of the dataset
case  <- 1:10
x1    <- c(1,1,0,1,0,0,1,0,1,0)
x2    <- c(0,1,0,1,0,1,1,0,1,0)
x3    <- c(0,1,1,1,0,1,1,0,1,0)

D <- data.frame(case, x1, x2, x3)
D

case,x1,x2,x3
<int>,<dbl>,<dbl>,<dbl>
1,1,0,0
2,1,1,1
3,0,0,1
4,1,1,1
5,0,0,0
6,0,1,1
7,1,1,1
8,0,0,0
9,1,1,1


Where each record (row) is called $case$.
In particular we want to find dependencies among the variables Z = {x1, x2, x3} in D. 

The idea is to build an algorithm that provides the most probable (higher posterior) Bayesian belief network structure B given D and some assumptions about our prior probabilistic knowledge on the domain.

Where A belief network is defined by B =($B_s$, $B_p$).

  add scheme 


#### assumptions for computing $P(B_s, D)$
* 1- The database variables, which we denote as Z, are discrete.
* 2-  Cases occur independently, given a belief-network model.
* 3- There are no cases that have variables with missing values.
* 4- The likelihhod function f(BelBs) is uniform.

#### notation
* m = cases in D
* $parents$: variables related to $x_i$
* $\pi_i$ = set of parents related to $x_i$
* If a node has no parents, then a prior-probability function $P(x_i)$ is specified
* $w_{ij}$ a value of $\pi_i$
* $Z = \{x\}_i$, i = 1,..,n set of variables
* n = Z size 
* $v_i = (v_{i1},...,v_{ir_i})$ : possible value assignments
* $r_i$ = size of $v_i$
* $w_i = \{ w_{ij}, j = 1,..,r_j\}$ unique instantiation of $\pi_i$ relative to D. 
* $q_i$ = size of $w_i$ 
* $N_{ijk}$: number of cases in D in which variable $x_i$ has the value $v_{ik}$ and $\pi_i$ is instantiated as $w_{ij}$
#### in our problem: 

* m = 10
* n = 3
* Z = {$x_1$, $x_2$, $x_3$}
* $r_1$ = $r_2$ = $r_3$ = 2
* $v_i = (v_{i1},...,v_{ir_i})$ = $(v_{i1},v_{i2})$=(0,1) 
* $q_1$ = $q_2$ = $q_3$ = 2
* $w_i = (w_{i1},...,w_{iq_i})$ = $(w_{i1},w_{i2})$=(0,1) 
* $N_{111}$ = dove $x_1$ ha il valore 0 e i suoi parents anche (non ha parents) = 5
* $N_{112}$ = 5
* $N_{121}$ = 5
* $N_{122}$ = 5

* $N_{211}$ = 4 dove $x_2$ ha il valore 0 e i suoi parenti ($x_1$) anche
* $N_{212}$ = 1 dove $x_2$ ha il valore 1 e i suoi parenti ($x_1$) 0 
* $N_{221}$ = 1
* $N_{222}$ = 4

* $N_{311}$ = ($x_3$ = 0, $x_2$ = 0) = 4
* $N_{312}$ = ($x_3$ = 1, $x_2$ = 0) = 1
* $N_{321}$ = ($x_3$ = 0, $x_2$ = 1) = 0
* $N_{322}$ = ($x_3$ = 1, $x_2$ = 1) = 5

In [3]:
vi <- unique(D$x1)
ri <- length(vi)

N <- matrix(nrow = 2, ncol = 2)

N[1] <- 1
N[2] <- 2
N[3] <- 3
N[4] <- 4
N[row = 1, col = 2] <- 5
N
factorial(N[, col = 2])
prod(N[, col = 2])
prod(factorial(N[, col = 2]))

1,5
2,4


[1] 120  24

[1] 20

[1] 2880

In [4]:
# da usare dentro K2
# gli passo tipo x2 e x1
f <- function( xi, pi ){

    # possible values xi can take
    vi <- unique(xi)
    # number of possible values xi can take
    ri <- length(vi)
    wi <- unique(pi)
    qi <- length(wi)
    
    N <- matrix(nrow = ri, ncol = qi)
    f <- 1
    for (j in 1:qi){
        for (k in 1:ri){
            N[row = k, col = j] <- sum(xi == vi[k] & pi == wi[j])
            print(N)
        }
        f <- f*factorial(ri-1)*prod(factorial(N[, col = j]))/factorial(sum(N[, col = j])+ri-1)
    }
    
    return (f)
}


In [5]:
xi <- D$x1
pi <- c(D$x2, D$x3)
f(xi, pi)

     [,1] [,2]
[1,]    2   NA
[2,]   NA   NA
     [,1] [,2]
[1,]    2   NA
[2,]    7   NA
     [,1] [,2]
[1,]    2    8
[2,]    7   NA
     [,1] [,2]
[1,]    2    8
[2,]    7    3


[1] 1.402918e-06

In [13]:
score <- function(x.i, pi.i){
    '
    Function to compute the score: it represents the probability of the 
    database D given that the parents of x.i are pi.i

    Inputs:
        - x.i: list containing the i-th node of the database
        - pi.i: list of parents of node x.i

    Output:
        - probability of database given pi.i as parents of x.i
    '

    r.i <- length(unique(unlist(x.i)))
    
    # case without parents
    if(length(pi.i)==0){
        alpha <- table(x.i)
        N     <- sum(alpha)
        prob  <- factorial(r.i-1)/factorial(N+r.i-1)*prod(sapply(alpha,factorial))
    }

    # standard version with parents
    else{
        data  <- append(x.i,pi.i)
        alpha <- table(data)
        # recast in a matrix
        if (length(dim(alpha))>2){dim(alpha)<-c(r.i,length(alpha)/r.i)}
        # transpose
        alpha<-t(alpha)
        N.j <- rowSums(alpha)
        M.j <- apply(apply(alpha, c(1,2), factorial),1,prod)
        prob <- prod(apply(cbind(N.j,M.j),1, function(x) x[2]*factorial(r.i-1)/factorial(x[1] + r.i - 1)))
    }

    return(prob)
}



#score(D[2], list(NULL))
                           
for(j in 1:0) print(j)

[1] 1
[1] 0


In [14]:
K2 <- function(data, order=c(1,2,3), max.parents=2) {
    '
    K2 algorithm to learn the topology of a Bayesian Network.

    Input:
        - data: data.frame with n+1 columns (the first one is the case column)
        - order: vector with the ordering of the nodes in data
        - max.parents: upper bound on the number of parents a node may have
    Output:
        - 
    '
    # convert database to a list of vectors
    nodes <- as.list(data[, order+1])
    # number of nodes
    n <- length(nodes)
    
    # empty vector to store the parents of each node
    parents <- rep(NULL, 0)

    k2.score <- 0

    for( i in 1:n ){
        parents[i] <- c()
        p.old <- score(nodes[i], parents[i])
        
        ok.to.proceed <- TRUE
        pred_xi <- order[1:i-1]

        while(ok.to.proceed & length(parents)<max.parents){
            diff   <- append(parents, setdiff(pred_xi, parents))
            # create a list with the possible parent nodes to inspect
            unions <- nodes[diff]
            
            # node in Pred(x.i)-pi.i that maximizes score
            p.new <- 0
            id    <- 0
            
            if(length(unions)==0){
                z.new <- score(nodes[i], NULL)
                if(z.new>p.new){
                    p.new <- z.new
                    id    <- j
                }
            }
            else{
                for(j in 1:length(unions)){
                    z.new <- score(nodes[i], unions[j])
                    if(z.new>p.new){
                        p.new <- z.new
                        id    <- j
                    }
                }
            }

            if(p.new>p.old){
                p.old <- p.new
                pi.i  <- unions[id] 
            }
            else{
                ok.to.proceed<-FALSE
            }
        }
        k2.score <- k2.score + p.old
    }
}




K2(D)
# Input: A set of n nodes = 3 
#        an ordering on the nodes, (permutation of {1,2,3})
#        an upper bound u on the number of parents a node may have (n-1), 
#        a database D containing m cases
# Output: For each node, a printout of the parents of the node.}


In [21]:

data=D
order=c(1,2,3)
max.parents=2

nodes <- as.list(D[, order+1])
n <- length(nodes)
parents <- rep(NULL, 0)

for( i in 1:n ){
    parents[i] <- c()
    p.old <- score(nodes[i], parents[i])
    print(p.old)
}


[1] 0.0003607504
[1] 0.0003607504
[1] 0.0004329004


In [131]:
i=1
r=2
order=c(1,2,3)
nodes <- as.list(D[, order+1])
pii=nodes[c(2,3)]
data1 <- append(nodes[i],pii)
data1


alpha <- table(data1)
print(alpha)
if (length(dim(alpha))>2){dim(alpha)<-c(r,length(alpha)/r)}
alpha<-t(alpha)


alpha

N.j <- rowSums(alpha)
N.j
N<-sum(alpha)
N


M.j <- apply(apply(alpha, c(1,2), factorial),1,prod)
prob <- prod(apply(cbind(N.j,M.j),1, function(x) x[2]*factorial(r-1)/factorial(x[1] + r - 1)))
                   
M.j
prob
                   
cbind(N.j,M.j)

$x1
 [1] 1 1 0 1 0 0 1 0 1 0

$x2
 [1] 0 1 0 1 0 1 1 0 1 0

$x3
 [1] 0 1 1 1 0 1 1 0 1 0

, , x3 = 0

   x2
x1  0 1
  0 3 0
  1 1 0

, , x3 = 1

   x2
x1  0 1
  0 1 1
  1 0 4



     [,1] [,2]
[1,]    3    1
[2,]    0    0
[3,]    1    0
[4,]    1    4

[1] 4 0 1 5

[1] 10

[1]  6  1  1 24

[1] 0.0008333333

N.j,M.j
4,6
0,1
1,1
5,24


In [127]:
myscore <- function(i, pi, D){
    # number of possible ourcomes for node i
    r <- length(unique(D[,i]))  
    # generalized formula for no-parent case (from paper)
    if (length(pi) == 0){
        alpha <- table(D[i])
        N <- sum(alpha)
        prob <- factorial(r-1)/factorial(N+r-1)*prod(sapply(alpha,factorial))
    }
    # standard formula (from paper)
    else{ 
        D2 <- cbind(D[i],D[pi])
        print(D2)
        alpha <- table(D2)
        print(alpha)
        if (length(dim(alpha))>2){dim(alpha)<-c(r,length(alpha)/r)}
        print(alpha)
        alpha <- t(alpha)
        print(alpha)
        N.j <- rowSums(alpha)
        print(N.j)
        M.j <- apply(apply(alpha, c(1,2), factorial),1,prod)
        print(cbind(N.j,M.j))
        prob <- prod(apply(cbind(N.j,M.j),1, function(x) x[2]*factorial(r-1)/factorial(x[1] + r - 1))) 
    }
    return (prob)
}
                           
myscore(2, c(3,4), D)
                           
#c(D[3], D[4])

   x1 x2 x3
1   1  0  0
2   1  1  1
3   0  0  1
4   1  1  1
5   0  0  0
6   0  1  1
7   1  1  1
8   0  0  0
9   1  1  1
10  0  0  0
, , x3 = 0

   x2
x1  0 1
  0 3 0
  1 1 0

, , x3 = 1

   x2
x1  0 1
  0 1 1
  1 0 4

     [,1] [,2] [,3] [,4]
[1,]    3    0    1    1
[2,]    1    0    0    4
     [,1] [,2]
[1,]    3    1
[2,]    0    0
[3,]    1    0
[4,]    1    4
[1] 4 0 1 5
     N.j M.j
[1,]   4   6
[2,]   0   1
[3,]   1   1
[4,]   5  24


[1] 0.0008333333

In [153]:
parent=rep(NULL, 0)
nodes <- c(1,2,3)
pred_xi <- nodes[1:3-1]
pred_xi

diff <- setdiff(pred_xi, parent)
diff

unions <- lapply(diff, union, parent)
unions

new.parent <- unlist(unions[which.max(sapply(unions, function(x) score(x.i = D[2], c(D[3], D[4]))))])
new.parent

[1] 1 2

[1] 1 2

[[1]]
[1] 1

[[2]]
[1] 2

function (x, y, ...) 
UseMethod("union")
<bytecode: 0x5604fd6c1550>
<environment: namespace:generics>

[1] 1